In [1]:
import pandas as pd
!ls

EDA.ipynb                cleaned_data_ashish.pkl  preprocessed.pk
LDA.ipynb                extract_rtf.py           preprocessing_1.pk
RC_Media                 final_data.pk            sentiment.py
RC_Media_all_files       googleTrends.csv         sentiment_analysis.ipynb
RC_Media_all_files.zip   images                   test.ipynb
SentiWordNet_3.0.0.txt   old version
__pycache__              preprocess_data.ipynb


In [2]:
loaded_dataframe = pd.read_pickle("final_data.pk")
#clean_df=loaded_dataframe.dropna()
loaded_dataframe
clean_df = loaded_dataframe.dropna()
clean_df.head()

,title,publishing_date,publisher,leading_para,full_text,file
0,New car sales like canary in a coal mine,17 August 2018,The Age - Online,Car sales are dropping. July sales were down ...,The fact is we love cars and we all remember ...,Factiva-20190123-1357.rtf
1,New car sales like canary in a coal mine,17 August 2018,WAToday.com.au,Car sales are dropping. July sales were down ...,The fact is we love cars and we all remember ...,Factiva-20190123-1357.rtf
2,Taking on major banks isn't on Australia Post...,16 August 2018,Australian Broadcasting Corporation News,The spotlight on banking misconduct at the ro...,Mr Holgate told the ABC's AM program that the...,Factiva-20190123-1357.rtf
3,Banking royal commission live: superannuati...,16 August 2018,The Australian - Online,ASIC flags criminal action. The Royal Commiss...,Mr Hodge begins his ‘brief’ closing observati...,Factiva-20190123-1357.rtf
4,Katter pays the price for backing Anning's sp...,17 August 2018,Canberra Times,A backlash against federal MP Bob Katter is s...,The far north Queensland MP has been shunned ...,Factiva-20190123-1357.rtf


In [3]:
import nltk
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/ashish/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
 
lemmatizer = WordNetLemmatizer()
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
 
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0

 

In [5]:
from textblob import TextBlob
def get_polarity(text):
    pol = TextBlob(text).sentiment.polarity
    return pol
clean_df.loc[:,'textblog_polarity_ft'] = clean_df['full_text'].apply(get_polarity)
clean_df.loc[:,'swn_polarity_ft']=clean_df['full_text'].apply(swn_polarity)

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
clean_df.head()

,title,publishing_date,publisher,leading_para,full_text,file,textblog_polarity_ft,swn_polarity_ft
0,New car sales like canary in a coal mine,17 August 2018,The Age - Online,Car sales are dropping. July sales were down ...,The fact is we love cars and we all remember ...,Factiva-20190123-1357.rtf,0.187384,0
1,New car sales like canary in a coal mine,17 August 2018,WAToday.com.au,Car sales are dropping. July sales were down ...,The fact is we love cars and we all remember ...,Factiva-20190123-1357.rtf,0.187384,0
2,Taking on major banks isn't on Australia Post...,16 August 2018,Australian Broadcasting Corporation News,The spotlight on banking misconduct at the ro...,Mr Holgate told the ABC's AM program that the...,Factiva-20190123-1357.rtf,0.073329,1
3,Banking royal commission live: superannuati...,16 August 2018,The Australian - Online,ASIC flags criminal action. The Royal Commiss...,Mr Hodge begins his ‘brief’ closing observati...,Factiva-20190123-1357.rtf,0.128527,1
4,Katter pays the price for backing Anning's sp...,17 August 2018,Canberra Times,A backlash against federal MP Bob Katter is s...,The far north Queensland MP has been shunned ...,Factiva-20190123-1357.rtf,0.041333,0


In [9]:
cl = clean_df.loc[clean_df.swn_polarity_ft == 1, :].sample(20) #Random sample of positive titles
random=cl.sample(1)
pd.options.display.max_seq_items= 2000

In [37]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [45]:
def print_ner(text):
    doc = nlp(text)

# Analyze syntax
    #print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    #print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
    for entity in doc.ents:
        #print(entity.text, entity.label_)
        if entity.label_=='ORG':
            print(entity.text)
clean_df['full_text'].apply(print_ner)

Bluebelle
Blue Boy
Lincoln Continental
MG
Bluebelle
Blue Boy
Lincoln Continental
MG
ABC
Australia Post
Australia Post
— Commonwealth Bank
NAB
ANZ —
Australia Post
Australia Post's
Australia Post
the Australian Prudential Regulation Authority
Australia Post's
Deloitte Access Economics
Australia Post
Australia Post's
Holgate
MySuper
Sunsuper and Mercer
RSE
MySuper
’s
MySuper
DPP
ASIC
ASIC
DPP
ASIC
ASIC’s
MySuper
ADA
ASIC
ASIC
ASIC
ASIC
ASIC
ASIC
ASIC
ASIC
ASIC
ASIC
NAB
NULIS
NAB
ASIC
NAB
NULIS
NAB
ASIC
ASIC
ASIC
NAB
ASIC
NAB
ASIC
ASIC
ASIC
NAB
ASIC
ASIC
APRA
NULIS
ASIC
APRA
APRA
NAB
NULIS
NULIS
ASIC
MLC Nominees
NAB
ASIC
NAB’s
’s
ASIC
NAB
ASIC
NULIS
ASIC
Kell
ASIC
ASIC
ASIC
ASIC
NAB
ASIC
NULIS
ASIC
NAB
NULIS
EU
EU
EU
ASIC
NULIS
NAB
ASIC
NULIS
NAB
ASIC
EU
ASIC
’s
EU
ASIC
EU
EU
ASIC
ASIC
EU
ANZ
EU
ANZ
EU
CBA
ASIC
EU
CBA
EU
Mr Hodge
’s
Mr Mulally
ANZ
CBA
EU
CBA
ANZ
ASIC
ASIC
ANZ
ANZ
EU
Mr Mullaly
ASIC
EU
ANZ
EU
ASIC
ASIC
ANZ
ANZ
ASIC
ASIC
EU
ANZ.Commissioner Hayne
ASIC
ANZ
CBA
ASIC
CBA
ANZ


the Commonwealth Bank
ANZ
NAB
Westpac
AMP
ASIC
ASIC
ASIC
ASIC
Commonwealth
ASIC
APRA
the Australian Securities and Investments Commission
APRA
APRA
APRA
APRA
’s
APRA
’s
NULIS
NAB
CBA
Colonial First State
MySuper
Treasurer Scott Morrison
Treasurer
APRA
Mr Byres
APRA
House of Representatives
the Reserve Bank
the Council of Financial Regulators
APRA
ASIC
Treasury
Lowe
ASIC
AMP
ASIC
the National Australia Bank
ASIC
NAB
Kell
the Financial Services Royal Commission
Australian Prudential Regulation Authority
APRA
the Commonwealth Bank
Colonial First State
Rowell
APRA
Colonial First State
My Super
Colonial First State
Colonial First State
APRA
APRA
AMP
Rowell
ARPA
APRA
APRA
the Productivity Commission
The Australian Securities and Investments Commission
ASIC
ASIC
ASIC
ASIC
ASIC
Commonwealth
AMP
ASIC
ASIC
National Australia Bank's
NULIS
NAB
ASIC
ASIC
NAB
ASIC
White Australia Policy
Liberal National
Entsch
Labor
CFMMEU
Labor
Labor
Parliament
the Labor Party
KAP
Labor
KAP
Labor
arrival".However
L

Ten News
the RHT Family Trust
the RHT Family Trust
Family First
the High Court
AAP
Sky’s David Speers
National Party
the House of Representatives
Sky News
’s
AGL Energy’s
Liddell
Energy
Liddell
AGL
Shorten
the House of Representatives
Labor
NEG
Shorten
the House of Representatives
DixerLabor
Home Affairs
Dixer
the Labor Party
Labor
Coalition
Labor
Commonwealth
Senate
Nation’s
Senate
Labor
Labor
Labor
Labor
Parliament?”Mr Turnbull
the House of Representatives.“We
NEG.In
Agriculture
Indigenous Affairs
Nationals
NEG
the House of Representatives
Nationals
’m
ACCC
the National Party
the National Party
Labor’s
Labor
Labor
Nationals
Liberals
the Liberal Party
the National Party
Nationals
Sky News
Nationals
NEG
Labor
TurnbullBarnaby Joyce
Sky News Australia.“We
Liberal
the Malcolm Turnbull
Liberal National
Liberal
Opposition
the National Energy Guarantee
Turnbull’s
the Liberal Party
Labor
Mr Turnbull’s
Labor
Coalition
week.“Malcolm
Entsch
Entsch
Morrison
Liberal
the House of Representatives
th

KeyboardInterrupt: 

In [ ]:
#Where the Australia Post is coming from
#What tokens are being filtered in LDA